## Agents and Tools

### Agents: Agents enable LLMs to work with tools that perform various tasks
To use agents, we require three things:
- A base LLM,
- A tool that we will be interacting with,
- An agent to control the interaction.

### In the following example, we will use the Math tool to perform some simple math operations

In [1]:
from dotenv import load_dotenv
from langchain_openai import OpenAI
load_dotenv()

/Users/abhishekpratap.singh/DataspellProjects/Gen-AI---RAG-Application-Development-using-LangChain/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


False

In [12]:
import openai
print("OpenAI package version:", openai.__version__)


OpenAI package version: 1.63.2


In [22]:
# from openai import OpenAI
# llm = OpenAI()
# completion = client.chat.completions.create(
#     model="gpt-4o",
#     store=True,
#     messages=[
#         {"role": "user", "content": "write a haiku about ai"}
#     ]
# )
#
# print(completion.choices[0].message.content)







In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMMathChain
from langchain.tools import Tool

# Initialize the LLM
llm = ChatOpenAI(model_name="gpt-4-turbo", temperature=0)

# Correct way to initialize LLMMathChain
llm_math = LLMMathChain.from_llm(llm)

# Define the calculator tool
math_tool = Tool(
    name="Calculator",
    func=llm_math.invoke,  # Updated to use .invoke instead of .run
    description="Useful for when you need to answer questions about math."
)

# List of tools
tools = [math_tool]

# Test the tool
question = "What is 37593 times 67?"
result = math_tool.func({"question": question})
print("Result:", result)



Result: {'question': 'What is 37593 times 67?', 'answer': 'Answer: 2518731'}


## Now create an Agent
#### Zero-shot ReAct
The Zero-shot ReAct Agent is a language generation model that can create realistic contexts even without being trained on specific data. It can be used for various tasks such as generating creative text formats, language translation, and generating different types of creative content.

In [6]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [7]:
zero_shot_agent("What is root over 25")

/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use a calculator to find the square root of 25
Action: Calculator
Action Input: √25
Observation: Answer: 5.0
Thought: I now know the final answer
Final Answer: 5.0

> Finished chain.


{'input': 'What is root over 25', 'output': '5.0'}

In [8]:
problem = """
        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen 
        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. 
        What is the total area of your house?
        """
zero_shot_agent(problem)



> Entering new AgentExecutor chain...
 I need to calculate the area of each room and then add them together to get the total area of the house.
Action: Calculator
Action Input: 5 * 5
Observation: Answer: 25
Thought: This is the area of one bedroom.
Action: Calculator
Action Input: 7 * 6
Observation: Answer: 42
Thought: This is the area of the drawing cum open kitchen.
Action: Calculator
Action Input: 3 * 2
Observation: Answer: 6
Thought: This is the area of the balcony.
Action: Calculator
Action Input: 25 + 25 + 42 + 6
Observation: Answer: 98
Thought: This is the total area of the house.
Final Answer: The total area of the house is 98 square metres.

> Finished chain.


{'input': '\n        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen \n        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. \n        What is the total area of your house?\n        ',
 'output': 'The total area of the house is 98 square metres.'}

## Using mutiple Tools
In the following example, we will be using the Duck Duck Go search API as a Tool combined with the Math Tool
We will then give the Agent a problem that initiates a search first followed by a Calculation.

In [28]:
# Define the Tools

from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(
    func=search.run,
    name="Search",
    description="useful for when you need to search the internet for information"
)

llm_math_chain = LLMMathChain.from_llm(llm)

math_tool = Tool.from_function(
    func=llm_math_chain.run,
    name="Calculator",
    description="Useful for when you are asked to perform math calculations"
)

In [30]:
# Define the Agent

tools = [search_tool, math_tool]

#  agent = initialize_agent(
#     tools,
#     llm,
#     agent="zero-shot-react-description",
#     verbose=True
# )

from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

/var/folders/p2/shr4yl517qjchhm4bzmn77rc0000gp/T/ipykernel_9720/39232151.py:14: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  zero_shot_agent = initialize_agent(


In [33]:
from langgraph.prebuilt import create_react_agent

react_agent = create_react_agent(llm, tools)


NotImplementedError: 

In [25]:
# Run the Agent with the problem

agent.run("""Get Microsoft Stock Price taken from Google Finance and display in both USD and INR values""")



> Entering new AgentExecutor chain...
 I should use the search tool to find the current Microsoft stock price on Google Finance.
Action: Search
Action Input: "Microsoft stock price Google Finance"
Observation: Microsoft. Market Cap. Today's Change. (1.45%) $5.92. Current Price. $413.64. Price as of February 29, 2024, 4:00 p.m. ET. You're reading a free article with opinions that may differ from The ... Get a real-time Microsoft Corporation (MSFT) stock price quote with breaking news, financials, statistics, charts and more. ... Yahoo Finance. Exclusive: Google steps up Microsoft criticism, warns of rival's monopoly in cloud. 3.00 (0.74%) Ex-Dividend Date. 2024-02-14. Analyst rating and price target. MSFT is rated "Strong Buy" with a 12-month target price of $465.63, a ‎12.57% upside from the current price of $413.64 ... Get the latest Microsoft stock price NASDAQ: MSFT stock rating and detailed information including MSFT news, historical charts and real-time prices. View Microsoft Co

'The current Microsoft stock price is $413.64 USD and 34,382.50 INR.'

## Create a custom tool

In [29]:
from langchain.tools import BaseTool
from langchain_core.output_parsers import JsonOutputParser
from langchain.tools import tool

@tool("JSON_Response", return_direct=True)
def StructuredResponseTool(question: str):
    """
    use this tool to send a prompt and get a JSON returned 
    with three fields - Topic, Question_Details and Detailed_Response
    """
    json_prompt = PromptTemplate.from_template(
    """Return a JSON object with an `answer` key that answers the following question: {question}. 
        The JSON object will have three fields - Topic, Question_Details and Detailed_Response"""
    )

    model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)

    json_parser = JsonOutputParser()
    json_chain = json_prompt | model | json_parser
    x = json_chain.invoke({"question": question})
    return x

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [31]:
from langchain.agents import initialize_agent

tools = [StructuredResponseTool]

# initialize agent with tools
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

In [32]:
zero_shot_agent.run(input="Which is the top football team in the world and how, the response should have three fields in the JSON - topic of the question, the question and the detailed response")



> Entering new AgentExecutor chain...
 I should use the JSON_Response tool to get the answer
Action: JSON_Response
Action Input: "Which is the top football team in the world and how"
Observation: {'Topic': 'Football', 'Question_Details': 'Which is the top football team in the world and how?', 'Detailed_Response': "The top football team in the world is subjective and can vary depending on personal opinions. However, according to FIFA's current world rankings, the top football team in the world is Belgium. This is determined by a points system based on the team's performance in international matches. Belgium has consistently performed well in recent years and has a talented squad of players, including Kevin De Bruyne, Eden Hazard, and Romelu Lukaku. They also have a strong team strategy and have been successful in major tournaments such as the World Cup and European Championship. Overall, Belgium's consistent success and strong team make them the top football team in the world accordin

{'Topic': 'Football',
 'Question_Details': 'Which is the top football team in the world and how?',
 'Detailed_Response': "The top football team in the world is subjective and can vary depending on personal opinions. However, according to FIFA's current world rankings, the top football team in the world is Belgium. This is determined by a points system based on the team's performance in international matches. Belgium has consistently performed well in recent years and has a talented squad of players, including Kevin De Bruyne, Eden Hazard, and Romelu Lukaku. They also have a strong team strategy and have been successful in major tournaments such as the World Cup and European Championship. Overall, Belgium's consistent success and strong team make them the top football team in the world according to FIFA's rankings."}